In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math,copy
import os
from scipy import optimize
%matplotlib inline
np.set_printoptions(precision=2)

In [2]:
rain_train_data = pd.read_csv("weather_train.csv")
display(rain_train_data)


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,01-12-2008,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,02-12-2008,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,03-12-2008,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,04-12-2008,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,05-12-2008,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52057,21-06-2017,Tuggeranong,-1.2,15.1,0.4,NaN,NaN,WSW,15.0,NaN,...,100.0,38.0,1028.8,1026.3,NaN,NaN,-0.4,14.7,No,No
52058,22-06-2017,Tuggeranong,-0.4,14.0,0.0,NaN,NaN,N,24.0,NaN,...,83.0,44.0,1029.5,1025.0,NaN,NaN,6.0,13.1,No,No
52059,23-06-2017,Tuggeranong,-3.5,13.4,0.0,NaN,NaN,NW,33.0,SW,...,100.0,38.0,1022.0,1016.1,NaN,NaN,0.1,12.9,No,No
52060,24-06-2017,Tuggeranong,0.1,14.5,0.0,NaN,NaN,NW,35.0,N,...,64.0,43.0,1017.1,1015.4,NaN,NaN,9.5,13.1,No,No


In [3]:
rain_train_data.columns


Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

In [4]:
rain_train_data["RainTomorrow"].replace("No",0,inplace=True)
rain_train_data["RainTomorrow"].replace("yes",1,inplace=True)
display(rain_train_data)



,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,01-12-2008,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0
1,02-12-2008,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0
2,03-12-2008,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0
3,04-12-2008,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,0
4,05-12-2008,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52057,21-06-2017,Tuggeranong,-1.2,15.1,0.4,NaN,NaN,WSW,15.0,NaN,...,100.0,38.0,1028.8,1026.3,NaN,NaN,-0.4,14.7,No,0
52058,22-06-2017,Tuggeranong,-0.4,14.0,0.0,NaN,NaN,N,24.0,NaN,...,83.0,44.0,1029.5,1025.0,NaN,NaN,6.0,13.1,No,0
52059,23-06-2017,Tuggeranong,-3.5,13.4,0.0,NaN,NaN,NW,33.0,SW,...,100.0,38.0,1022.0,1016.1,NaN,NaN,0.1,12.9,No,0
52060,24-06-2017,Tuggeranong,0.1,14.5,0.0,NaN,NaN,NW,35.0,N,...,64.0,43.0,1017.1,1015.4,NaN,NaN,9.5,13.1,No,0


In [5]:
mean_1 = np.mean(rain_train_data["Cloud9am"])
mean_2 = np.mean(rain_train_data["Cloud3pm"])
mean_3 = np.mean(rain_train_data["Evaporation"])
mean_4 = np.mean(rain_train_data["Sunshine"])
mean_5 = np.mean(rain_train_data["WindGustSpeed"])
mean_6 = np.mean(rain_train_data["WindSpeed9am"])
mean_7 = np.mean(rain_train_data["WindSpeed3pm"])
mean_8 = np.mean(rain_train_data["Humidity3pm"])
mean_9 = np.mean(rain_train_data["Pressure9am"])
mean_10 = np.mean(rain_train_data["Pressure3pm"])
mean_11 = np.mean(rain_train_data["Temp9am"])
mean_12 = np.mean(rain_train_data["Temp3pm"])
mean_13 = np.mean(rain_train_data["MinTemp"])
mean_14 = np.mean(rain_train_data["MaxTemp"])
mean_15 = np.mean(rain_train_data["Rainfall"])

rain_train_data["Cloud9am"].fillna(value = mean_1,inplace=True)
rain_train_data["Cloud3pm"].fillna(value = mean_2,inplace=True)
rain_train_data["Evaporation"].fillna(value = mean_3,inplace=True)
rain_train_data["Sunshine"].fillna(value = mean_4,inplace=True)
rain_train_data["WindGustSpeed"].fillna(value = mean_5,inplace=True)
rain_train_data["WindSpeed9am"].fillna(value = mean_6,inplace=True)
rain_train_data["WindSpeed3pm"].fillna(value = mean_7,inplace=True)
rain_train_data["Humidity3pm"].fillna(value = mean_8,inplace=True)
rain_train_data["Pressure9am"].fillna(value = mean_9,inplace=True)
rain_train_data["Pressure3pm"].fillna(value = mean_10,inplace=True)
rain_train_data["Temp9am"].fillna(value = mean_11,inplace=True)
rain_train_data["Temp3pm"].fillna(value = mean_12,inplace=True)
rain_train_data["MinTemp"].fillna(value = mean_13,inplace=True)
rain_train_data["MaxTemp"].fillna(value = mean_14,inplace=True)
rain_train_data["Rainfall"].fillna(value = mean_15,inplace=True)
display(rain_train_data)


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,01-12-2008,Albury,13.4,22.9,0.6,5.52876,7.568706,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.00000,4.487216,16.9,21.8,No,0
1,02-12-2008,Albury,7.4,25.1,0.0,5.52876,7.568706,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,4.43216,4.487216,17.2,24.3,No,0
2,03-12-2008,Albury,12.9,25.7,0.0,5.52876,7.568706,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,4.43216,2.000000,21.0,23.2,No,0
3,04-12-2008,Albury,9.2,28.0,0.0,5.52876,7.568706,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,4.43216,4.487216,18.1,26.5,No,0
4,05-12-2008,Albury,17.5,32.3,1.0,5.52876,7.568706,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.00000,8.000000,17.8,29.7,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52057,21-06-2017,Tuggeranong,-1.2,15.1,0.4,5.52876,7.568706,WSW,15.0,NaN,...,100.0,38.0,1028.8,1026.3,4.43216,4.487216,-0.4,14.7,No,0
52058,22-06-2017,Tuggeranong,-0.4,14.0,0.0,5.52876,7.568706,N,24.0,NaN,...,83.0,44.0,1029.5,1025.0,4.43216,4.487216,6.0,13.1,No,0
52059,23-06-2017,Tuggeranong,-3.5,13.4,0.0,5.52876,7.568706,NW,33.0,SW,...,100.0,38.0,1022.0,1016.1,4.43216,4.487216,0.1,12.9,No,0
52060,24-06-2017,Tuggeranong,0.1,14.5,0.0,5.52876,7.568706,NW,35.0,N,...,64.0,43.0,1017.1,1015.4,4.43216,4.487216,9.5,13.1,No,0


In [6]:
rain_train_data = rain_train_data.drop(["Date","Location","WindGustDir","WindDir9am","WindDir3pm","RainToday"],axis=1)
display(rain_train_data)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow
0,13.4,22.9,0.6,5.52876,7.568706,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.00000,4.487216,16.9,21.8,0
1,7.4,25.1,0.0,5.52876,7.568706,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,4.43216,4.487216,17.2,24.3,0
2,12.9,25.7,0.0,5.52876,7.568706,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,4.43216,2.000000,21.0,23.2,0
3,9.2,28.0,0.0,5.52876,7.568706,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,4.43216,4.487216,18.1,26.5,0
4,17.5,32.3,1.0,5.52876,7.568706,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.00000,8.000000,17.8,29.7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52057,-1.2,15.1,0.4,5.52876,7.568706,15.0,0.0,7.0,100.0,38.0,1028.8,1026.3,4.43216,4.487216,-0.4,14.7,0
52058,-0.4,14.0,0.0,5.52876,7.568706,24.0,0.0,15.0,83.0,44.0,1029.5,1025.0,4.43216,4.487216,6.0,13.1,0
52059,-3.5,13.4,0.0,5.52876,7.568706,33.0,6.0,13.0,100.0,38.0,1022.0,1016.1,4.43216,4.487216,0.1,12.9,0
52060,0.1,14.5,0.0,5.52876,7.568706,35.0,9.0,17.0,64.0,43.0,1017.1,1015.4,4.43216,4.487216,9.5,13.1,0


In [7]:
x_rain_train_data = rain_train_data.drop(["RainTomorrow"],axis=1)
y_rain_train_data = rain_train_data["RainTomorrow"]
display(x_rain_train_data)

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
0,13.4,22.9,0.6,5.52876,7.568706,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.00000,4.487216,16.9,21.8
1,7.4,25.1,0.0,5.52876,7.568706,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,4.43216,4.487216,17.2,24.3
2,12.9,25.7,0.0,5.52876,7.568706,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,4.43216,2.000000,21.0,23.2
3,9.2,28.0,0.0,5.52876,7.568706,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,4.43216,4.487216,18.1,26.5
4,17.5,32.3,1.0,5.52876,7.568706,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.00000,8.000000,17.8,29.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52057,-1.2,15.1,0.4,5.52876,7.568706,15.0,0.0,7.0,100.0,38.0,1028.8,1026.3,4.43216,4.487216,-0.4,14.7
52058,-0.4,14.0,0.0,5.52876,7.568706,24.0,0.0,15.0,83.0,44.0,1029.5,1025.0,4.43216,4.487216,6.0,13.1
52059,-3.5,13.4,0.0,5.52876,7.568706,33.0,6.0,13.0,100.0,38.0,1022.0,1016.1,4.43216,4.487216,0.1,12.9
52060,0.1,14.5,0.0,5.52876,7.568706,35.0,9.0,17.0,64.0,43.0,1017.1,1015.4,4.43216,4.487216,9.5,13.1


# Normalizing the features

In [8]:
m,n = x_rain_train_data.shape
x_features = x_rain_train_data.columns

for i in range(n):
    diff_1 = (x_rain_train_data.iloc[:,i])-min(x_rain_train_data.iloc[:,i])
    diff_2 = max(x_rain_train_data.iloc[:,i]) - min(x_rain_train_data.iloc[:,i])
    z_norm = diff_1/diff_2
    x_rain_train_data[x_features[i]] = z_norm
    

In [9]:
x_rain_train_data

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
0,0.569921,0.454139,0.001617,0.038129,0.536788,0.289062,0.153846,0.289157,0.701031,0.212121,0.452579,0.477080,0.888889,0.560902,0.490196,0.439189
1,0.411609,0.503356,0.000000,0.038129,0.536788,0.289062,0.030769,0.265060,0.422680,0.242424,0.500832,0.488964,0.492462,0.560902,0.497549,0.495495
2,0.556728,0.516779,0.000000,0.038129,0.536788,0.304688,0.146154,0.313253,0.360825,0.292929,0.450915,0.504244,0.492462,0.250000,0.590686,0.470721
3,0.459103,0.568233,0.000000,0.038129,0.536788,0.132812,0.084615,0.108434,0.432990,0.151515,0.617304,0.573854,0.492462,0.560902,0.519608,0.545045
4,0.678100,0.664430,0.002695,0.038129,0.536788,0.265625,0.053846,0.240964,0.814433,0.323232,0.504160,0.458404,0.777778,1.000000,0.512255,0.617117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52057,0.184697,0.279642,0.001078,0.038129,0.536788,0.062500,0.000000,0.084337,1.000000,0.373737,0.803661,0.803056,0.492462,0.560902,0.066176,0.279279
52058,0.205805,0.255034,0.000000,0.038129,0.536788,0.132812,0.000000,0.180723,0.824742,0.434343,0.815308,0.780985,0.492462,0.560902,0.223039,0.243243
52059,0.124011,0.241611,0.000000,0.038129,0.536788,0.203125,0.046154,0.156627,1.000000,0.373737,0.690516,0.629881,0.492462,0.560902,0.078431,0.238739
52060,0.218997,0.266219,0.000000,0.038129,0.536788,0.218750,0.069231,0.204819,0.628866,0.424242,0.608985,0.617997,0.492462,0.560902,0.308824,0.243243


In [25]:
x=np.array(x_rain_train_data)
y=np.array(y_rain_train_data)

In [26]:
def sigmoid(z):
    
    g = 1/(1+np.exp(-z))

    return g

In [27]:
def compute_cost_logistic(X, y, w):
    m = x.shape[0]
    # z = wT*x + C
    z =  np.dot(w, x.T)
    h = sigmoid(z)
    J = (1/m)*np.sum( -y*np.log(h) - (1-y)*np.log(1-h) )    
    return J

In [28]:
m,n = x_rain_train_data.shape
#initialize the parameter
np.random.seed(1)
#w_initial =np.random.rand(5)
w_initial =np.random.uniform(-10,10,n)
print(w_initial)

[ -1.66   4.41 -10.    -3.95  -7.06  -8.15  -6.27  -3.09  -2.06   0.78
  -1.62   3.7   -5.91   7.56  -9.45   3.41]


In [23]:
cost_history = []
# callback function
def callback(w):
    J = compute_cost_logistic(x, y,w_initial)
    cost_history.append(J)

In [29]:

res = optimize.minimize(compute_cost_logistic,w_initial,args=(x, y),method = 'TNC', options = {'maxiter' : 1000},callback=callback)



ValueError: shapes (52062,) and (16,52062) not aligned: 52062 (dim 0) != 16 (dim 0)

In [ ]:
i = np.linspace(1,len(cost_history),num=len(cost_history))
plt.plot(i,cost_history)
plt.ylabel("cost_history")
plt.xlabel("iterations")
plt.show()

In [ ]:
optimized_w = res.x

def predict(w,x,threshold):
    z =  np.matmul(w,np.transpose(x))
    h = sigmoid(z)
    h = np.where(h > threshold , 1, 0)
    return h

predictions_train = predict(optimized_w,x_a,0.5)
predictions_train, sum(predictions_train)